<a href="https://colab.research.google.com/github/rohailkhan/Coursera_ML1_Python/blob/master/Assignment%2B4%20Real.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [6]:
def get_list_of_university_towns():
    townslist1='/content/drive/My Drive/Coursera Michigan Python/university_towns.txt'
    university_towns2 = []
    states2=[]
    with open(townslist1) as file:
      for line in file:
        if '[edit]' in line:
    # Remember this `state` until the next is found
          states2.append(line)
        else:
    # Otherwise, we have a city; keep `state` as last-seen
          university_towns2.append(line)
    zipped_list=zip(states2,university_towns2)
    towns_df=pd.DataFrame(zipped_list,columns=['State', 'RegionName'])
    def get_citystate(item):
      if '(' in item:
          return item[:item.find('(')]
      elif '[' in item:
          return item[:item.find('[')]
      else:
          return item
    towns_df['State']=towns_df['State'].apply(get_citystate)
    towns_df['RegionName']=towns_df['RegionName'].apply(get_citystate)
         
    
    return towns_df

In [ ]:
get_list_of_university_towns()

# Recessin Start

In [17]:
gdplev1='/content/drive/My Drive/Coursera Michigan Python/gdplev.xls'

In [18]:
def get_gdp_time_series():
    '''Returns a DataFrame of GDP over time from the gdplev.xls file.'''
        
    r = (pd.read_excel(gdplev1,
                      skiprows=7)
        .drop(columns=['Unnamed: 0','Unnamed: 1','Unnamed: 2',
                       'Unnamed: 3','Unnamed: 5','Unnamed: 7'])
        .rename(columns={'Unnamed: 4':'YearQ',
                        'Unnamed: 6':'GDP'}))
    r = r[r['YearQ']>'2000']
    
    return r

In [33]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    r = get_gdp_time_series()
    
    rec_start = r[(r['GDP'] > r['GDP'].shift(-1))&
                  (r['GDP'].shift(-1) > r['GDP'].shift(-2))]
    
    return rec_start.iloc[0,0]

In [34]:
get_recession_start()
# answer should be 2008q3

'2008q2'

In [21]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    r = get_gdp_time_series()
    rec_start = get_recession_start()
    r = r[r['YearQ'] >= rec_start].copy()
    
    rec_end = r[(r['GDP'] > r['GDP'].shift(1))&
                (r['GDP'].shift(1) > r['GDP'].shift(2))].copy()
    
    return rec_end.iloc[0,0]

In [22]:
get_recession_end()
# answer should be '2009q4'

'2009q4'

In [36]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    r = get_gdp_time_series()
    rec_start = get_recession_start()
    rec_end = get_recession_end()
    r = r[(r['YearQ'] >= rec_start)&
          (r['YearQ'] <= rec_end)].copy()
    
    rec_bottom = r[r['GDP']==r['GDP'].min()].copy()
    
    return rec_bottom.iloc[0,0]

 #Answer should be 2009q2
 get_recession_bottom()

'2009q2'

In [ ]:
# Answer should be 2009q2

In [107]:
City_Zhvi_AllHomes='/content/drive/My Drive/Coursera Michigan Python/City_Zhvi_AllHomes.csv'

In [108]:
df = pd.read_csv(City_Zhvi_AllHomes, header=0,sep=",")
df.head(2)

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,CountyName,1996-01-31,1996-02-29,1996-03-31,1996-04-30,1996-05-31,1996-06-30,1996-07-31,1996-08-31,1996-09-30,1996-10-31,1996-11-30,1996-12-31,1997-01-31,1997-02-28,1997-03-31,1997-04-30,1997-05-31,1997-06-30,1997-07-31,1997-08-31,1997-09-30,1997-10-31,1997-11-30,1997-12-31,1998-01-31,1998-02-28,1998-03-31,1998-04-30,1998-05-31,1998-06-30,1998-07-31,1998-08-31,...,2016-12-31,2017-01-31,2017-02-28,2017-03-31,2017-04-30,2017-05-31,2017-06-30,2017-07-31,2017-08-31,2017-09-30,2017-10-31,2017-11-30,2017-12-31,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31
0,6181,0,New York,City,NY,NY,New York-Newark-Jersey City,Queens County,196258.0,195693.0,195383.0,194836.0,194652.0,194520.0,194447.0,194313.0,194271.0,194341.0,194496.0,194755.0,195144.0,196135.0,197185.0,198143.0,198601.0,199185.0,199877.0,200789.0,201707.0,202620.0,203785.0,204995.0,206282.0,206895.0,207400.0,207784.0,208585.0,209417.0,210201.0,211248.0,...,581449.0,585604.0,588436.0,591372.0,593522.0,597010.0,600522.0,605052.0,608956.0,612954.0,617462.0,621093.0,623901.0,626036.0,629857.0,633923.0,638221.0,641450.0,644658.0,646962.0,649745.0,652793.0,655131.0,657389.0,658632.0,659100.0,658771.0,659007.0,659422.0,659843.0,659421.0,659007.0,658239.0,656925.0,655613.0,654394.0,653930.0,653901.0,653565.0,652307.0
1,12447,1,Los Angeles,City,CA,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,185649.0,185625.0,185645.0,185706.0,185755.0,185852.0,186014.0,185977.0,185812.0,185482.0,185346.0,185219.0,184629.0,184056.0,183653.0,183835.0,184073.0,184301.0,184759.0,185678.0,186633.0,187795.0,188687.0,190038.0,191973.0,194272.0,196337.0,197771.0,199307.0,200699.0,201659.0,202835.0,...,630619.0,634128.0,637239.0,640855.0,643133.0,647743.0,651675.0,656294.0,660355.0,664284.0,667725.0,673738.0,679687.0,685331.0,691194.0,697740.0,702650.0,704730.0,706155.0,709156.0,711451.0,713262.0,714359.0,715685.0,715600.0,715717.0,713276.0,711206.0,709806.0,710747.0,712660.0,713807.0,715688.0,718245.0,721896.0,725180.0,730358.0,735910.0,744137.0,752508.0


In [ ]:
df.replace()

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    City_Zhvi_AllHomes='/content/drive/My Drive/Coursera Michigan Python/City_Zhvi_AllHomes.csv'
    hdata = pd.read_csv(City_Zhvi_AllHomes)
    hdata = hdata.drop(hdata.columns[[0]+list(range(3,51))],axis=1)
    #hdata2 = pd.DataFrame(hdata[['State','RegionName']])
    hdata2=hdata.loc[:,['State','RegionName']]
    for year in range(2000,2016):
        hdata2[str(year)+'q1'] = hdata[[str(year)+'-01',str(year)+'-02',str(year)+'-03']].mean(axis=1)
        hdata2[str(year)+'q2'] = hdata[[str(year)+'-04',str(year)+'-05',str(year)+'-06']].mean(axis=1)
        hdata2[str(year)+'q3'] = hdata[[str(year)+'-07',str(year)+'-08',str(year)+'-09']].mean(axis=1)
        hdata2[str(year)+'q4'] = hdata[[str(year)+'-10',str(year)+'-11',str(year)+'-12']].mean(axis=1)
    year = 2016    
    hdata2[str(year)+'q1'] = hdata[[str(year)+'-01',str(year)+'-02',str(year)+'-03']].mean(axis=1)
    hdata2[str(year)+'q2'] = hdata[[str(year)+'-04',str(year)+'-05',str(year)+'-06']].mean(axis=1)
    hdata2[str(year)+'q3'] = hdata[[str(year)+'-07',str(year)+'-08']].mean(axis=1)
    #hdata2 = hdata2.replace(mapper={'State':states},inplace=True)
    
    hdata2 = hdata2.set_index(['State','RegionName'])
    return hdata2

convert_housing_data_to_quarters()

In [134]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    
    '''
    City_Zhvi_AllHomes='/content/drive/My Drive/Coursera Michigan Python/City_Zhvi_AllHomes.csv'
   
    df_housing_data = pd.read_csv(City_Zhvi_AllHomes) 

    
    df_housing_data.drop(['Metro','CountyName','RegionID','SizeRank'],axis=1,inplace=True)
    #df_housing_data.drop(df.columns[[0, 1, 3]], axis=1) another way to drop columns by their number
    df_housing_data['State'] = df_housing_data['State'].map(states) #Full names of States instead of 2-digit short form
    df_housing_data.set_index(['State','RegionName'],inplace=True)  
    col = list(df_housing_data.columns)
    df_housing_data.drop(list(df_housing_data.columns)[:45],axis=1,inplace=True)
    
        
    def new_col_names():
        #generating the new coloumns names 
        years = list(range(2000,2017))
        quarters = ['q1','q2','q3','q4']
        quar_years = [] 
        for i in years:
            for x in quarters:
                quar_years.append((str(i)+x)) #append two strings and add to the list: quar_years
        return quar_years[:67] # we need columns for 2000q1 through 2016q3, don't want '2016q4' (the last element)
    
    # Q1 is January through March, Q2 is April through June, 
    # Q3 is July through September, Q4 is October through December.
    
    #qs is the quarters of the year
    qs = [list(df_housing_data.columns)[x:x+3] for x in range(0, len(list(df_housing_data.columns)), 3)]
    #creates a list of list, the inner lists are the 3 months of each Q
    # new columns based on quarters not year
    column_names = new_col_names() #changing the name of columns , This is a list, new_col_names() returns a list
    
    # zip(): Iterating through two lists at the same time
    for col,q in zip(column_names,qs):
        df_housing_data[col] = df_housing_data[q].mean(axis=1) #changing the name of columns of df_housing_data
        #creating a new column for df_housing_data: yearQ
    df_housing_data = df_housing_data[column_names]
          
    return df_housing_data


convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,2002q3,2002q4,2003q1,2003q2,2003q3,2003q4,2004q1,2004q2,2004q3,2004q4,2005q1,2005q2,2005q3,2005q4,2006q1,2006q2,2006q3,2006q4,2007q1,2007q2,2007q3,2007q4,2008q1,2008q2,2008q3,2008q4,2009q1,2009q2,2009q3,2009q4,2010q1,2010q2,2010q3,2010q4,2011q1,2011q2,2011q3,2011q4,2012q1,2012q2,2012q3,2012q4,2013q1,2013q2,2013q3,2013q4,2014q1,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
New York,New York,233094.666667,238732.666667,246395.666667,255003.000000,262691.666667,271436.333333,280304.000000,289046.666667,295521.000000,303292.333333,307906.333333,309308.333333,317637.666667,324954.666667,335090.333333,349041.333333,358477.666667,366755.666667,377364.333333,388348.000000,401696.333333,419612.666667,435378.333333,453253.666667,472749.000000,494132.000000,508452.666667,517543.000000,523867.666667,515742.666667,517316.000000,519836.000000,516904.666667,518599.000000,514909.000000,504238.666667,493909.666667,482655.333333,468563.000000,457774.666667,447647.000000,444330.666667,443404.000000,444245.000000,448730.000000,447303.666667,447268.333333,446624.333333,443790.666667,441998.666667,440679.666667,440832.666667,442397.333333,447135.333333,451386.666667,458080.000000,466499.666667,475229.000000,486669.666667,494606.666667,501934.333333,508733.666667,516028.333333,526105.666667,536209.666667,545984.666667,554140.333333
California,Los Angeles,228103.666667,232846.333333,238141.666667,244471.333333,251061.666667,257405.333333,263877.666667,270692.000000,277266.333333,285215.666667,294947.666667,306777.333333,321468.000000,337938.666667,352611.666667,366924.666667,384495.333333,404464.000000,430272.000000,461565.000000,489737.666667,514419.000000,538488.333333,561487.000000,591035.666667,617865.666667,635481.000000,650002.333333,652829.333333,651806.000000,648817.333333,643961.000000,629271.333333,607398.666667,579125.666667,535261.666667,500688.000000,469648.000000,448509.666667,437962.666667,436170.666667,441804.000000,441316.666667,444188.000000,435604.666667,425670.333333,420749.333333,411601.333333,405937.666667,401334.333333,399053.333333,402731.333333,411942.666667,428057.666667,451689.333333,479506.000000,508034.333333,523780.666667,529362.000000,532582.333333,534523.333333,541652.666667,554057.000000,562206.000000,570987.666667,583727.000000,595175.666667
Texas,Houston,104429.333333,105728.000000,106284.333333,106495.000000,107395.000000,108620.666667,108731.333333,108724.000000,108931.000000,109623.000000,110673.333333,112297.000000,113822.666667,114823.333333,115308.333333,115704.333333,116536.666667,117451.666667,120191.000000,123086.666667,125102.000000,127268.666667,128821.000000,128941.000000,128811.333333,128779.666667,129133.333333,130211.666667,131579.333333,132854.000000,133166.666667,133724.000000,134502.000000,134994.333333,133760.666667,131800.666667,130018.333333,127846.666667,127058.333333,127073.666667,127548.666667,127492.666667,127526.666667,127099.000000,124113.333333,122972.333333,120841.666667,118793.666667,118598.000000,117599.666667,117788.000000,118242.666667,119298.666667,120717.333333,123708.333333,127924.333333,131613.333333,135758.666667,138932.666667,141956.333333,146599.333333,150574.333333,154154.333333,157272.000000,158902.666667,160644.000000,162730.000000
Illinois,Chicago,152067.666667,154662.000000,159540.333333,164379.000000,168697.333333,173483.000000,177856.000000,182615.666667,187082.333333,191381.666667,195382.000000,199269.000000,203890.000000,208058.000000,212777.666667,216790.000000,221135.000000,226328.666667,231542.666667,237976.666667,244283.666667,250683.333333,256977.000000,264145.000000,271791.333333,278945.333333,284942.000000,288846.666667,290924.000000,292278.000000,293689.333333,290715.333333,288287.000000,285945.000000,280298.333333,273690.000000,264004.000000,252028.333333,239040.333333,229887.333333

In [144]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 

    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''

    import pandas as pd
    import numpy as np
    from scipy.stats import ttest_ind

    univ_towns = get_list_of_university_towns()
    univ_towns['univ_town'] = True

    # merge the housing data with university town DataFrames
    df = pd.merge(convert_housing_data_to_quarters(), univ_towns, how='outer', left_index=True, right_on=['State', 'RegionName'])
    df['univ_town'] = df['univ_town'].replace({np.NaN: False})

    # Get the recession quarters
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()

    # Parse the year and quarter of the recession quarters
    year_recession_start = int(recession_start[0:4])
    qtr_recession_start = int(recession_start[-1])
    year_recession_bottom = int(recession_bottom[0:4])
    qtr_recession_bottom = int(recession_bottom[-1])

    # get the columns to keep in the merged DataFrame
    cols_to_keep = ['State', 'RegionName', 'univ_town']
    qtrs_to_keep = []
    for i in range(year_recession_start, year_recession_bottom+1):
        for j in range(1, 5):
            if (i == year_recession_start and j < qtr_recession_start)\
                    or (i == year_recession_bottom and j > qtr_recession_bottom):
                pass
            else:
                qtrs_to_keep.append(str(i) + 'q' + str(j))
    df = df[cols_to_keep + qtrs_to_keep]

    # Compute the price_ratio
    df['price_ratio'] = df[recession_bottom] - df[recession_start]

    # t-test to determine if there is a difference between university and non-university towns
    univ_town_price_ratio = df[df['univ_town'] == True]['price_ratio']
    non_univ_town_price_ratio = df[df['univ_town'] == False]['price_ratio']
    st, p = ttest_ind(univ_town_price_ratio, non_univ_town_price_ratio, nan_policy='omit',)

    # get different and better values
    different = False
    if p < 0.01:
        different = True

    # determine which type of town is better
    better = ""
    if univ_town_price_ratio.mean() > non_univ_town_price_ratio.mean():
        better = "university town"
    else:
        better = "non-university town"

    return (different, p, better)

In [ ]:
run_ttest()

In [154]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Returns the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
     
    unitowns = get_list_of_university_towns()
    bottom = get_recession_bottom()
    start = get_recession_start()
    housing_data = convert_housing_data_to_quarters()
    # Selecting the quarter before the recession
    bstart = housing_data.columns[housing_data.columns.get_loc(start) - 1]
        
    """ 
    The formula for price ratio is :
    price_ratio=quarter_before_recession/recession_bottom
    """
    housing_data['ratio'] =  housing_data[bstart] / housing_data[bottom]
    housing_data = housing_data[[bottom, bstart, 'ratio']]
    housing_data = housing_data.reset_index()
    
    unitowns_hdata = pd.merge(housing_data,unitowns,how='inner',on=['State','RegionName'])
    unitowns_hdata['IsUniversityTown'] = True
    housing_data_complete = pd.merge(housing_data, unitowns_hdata, how='outer', on=['State','RegionName',bottom, bstart, 'ratio'])
    housing_data_complete['IsUniversityTown'] = housing_data_complete['IsUniversityTown'].fillna(False)
    
    university_towns = housing_data_complete[housing_data_complete['IsUniversityTown'] == True]
    non_universtiy_towns = housing_data_complete[housing_data_complete['IsUniversityTown'] == False]
    
    # Executing the Hypothesis test all while dropping the na values
    t,p = ttest_ind(university_towns['ratio'].dropna(), non_universtiy_towns['ratio'].dropna())
    # We are comparing the p value to 0.01 in our case
    different_prices = True if p<0.01 else False
    # Better contains the tag of the family of towns that have the better prices
    better_family = "university town" if university_towns['ratio'].mean() < non_universtiy_towns['ratio'].mean() else "non-university town"
    
    return (different_prices, p, better_family)

In [153]:
run_ttest()

(False, nan, 'non-university town')

In [155]:
    def run_ttest():
        housing=convert_housing_data_to_quarters()
        price=pd.DataFrame(housing[get_recession_bottom()]-housing[get_recession_start()], columns=['delta_price'])
        university_towns=get_list_of_university_towns()
        # method 1
        university_towns['test']=university_towns['State']+university_towns['RegionName']
        price=price.reset_index()
        price['Is_utowns']=price.apply(lambda x: x[0]+x[1] in list(university_towns['test']), axis=1)
        # method 2
        # university_towns['Is_utowns']=True
        # university_towns.set_index(['State', 'RegionName'], inplace=True)
        # price=pd.merge(price, university_towns, how='left', left_index=True, right_index=True).fillna(False)
        price_u=price[price['Is_utowns']==True]['delta_price']
        price_nu=price[price['Is_utowns']==False]['delta_price']
        statistic, pvalue=ttest_ind(price_u, price_nu, nan_policy='omit')
        if pvalue <0.01:
            different=True
        else:
            different=False
        if price_u.mean() > price_nu.mean(): #因为delta_price是用bottom减去start
            better='university town'
        else:
            better='non-university town'
        return (different, pvalue, better)
    run_ttest()

(False, nan, 'non-university town')

In [157]:


def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
     
    towns = get_list_of_university_towns()
    startdate = get_recession_start()
    bottomdate = get_recession_bottom()
    houses = convert_housing_data_to_quarters()
    
    houses = houses.reset_index()
    houses['recession_diff'] = houses[startdate] - houses[bottomdate]
    
    towns_houses = pd.merge(houses, towns, how='inner', on=['State', 'RegionName'])
    towns_houses['ctown'] = True
    houses = pd.merge(houses, towns_houses, how='outer', on = ['State', 'RegionName',
                                                              bottomdate, startdate, 
                                                              'recession_diff'])
    houses['ctown'] = houses['ctown'].fillna(False)
    unitowns = houses[houses['ctown'] == True]
    not_unitowns = houses[houses['ctown'] == False]
    
    t, p = ttest_ind(unitowns['recession_diff'].dropna(), not_unitowns['recession_diff'].dropna())
    different = True if p < 0.01 else False
    betters = "university town" if unitowns['recession_diff'].mean() < not_unitowns['recession_diff'].mean() else "non-university town"
    
    return different,p,betters

run_ttest()



(False, nan, 'non-university town')